In [6]:
from pylab import imshow, show, get_cmap
from numpy import random
import cv2 as cv
import numpy as np
from numpy import linalg
import pandas as pd
import numpy as np
from PIL import Image
import operator
import xarray as xr
import tensorflow as tf
from skimage.measure import label

## Part 1: Generating Dataset and Labels

In [2]:
# Create random dataset of 12x12 binary array

In [3]:
# Lets generate 5,000 random 4 by 4s

# rand_array is the key to convert to a black and white image object
# Binary array is the key to figure out euler number


len_data = 1
# This creates test_4, which is the nonbinary random array
i = 0
#return tensor of samples
test_list = []
while i <= (len_data):
    Z = random.random((12,12))
    Y = xr.DataArray(Z)

    test_list.append(Y)
    i += 1
test_4 = xr.concat(test_list, dim = "DataArray")
len(test_4)

2

In [4]:
# This creates a dataarray object of binary arrays

bin_list = []
for i in range (0, (len_data)):
    bin_pre_arr = np.array(test_4[i])
    img = Image.fromarray(bin_pre_arr, '1')
    #img.show()
    bin_arr = np.array(img)
    bin_arr = bin_arr.astype(np.int)
    bin_arr_xr = xr.DataArray(bin_arr)
    bin_list.append(bin_arr_xr)
    
test_4_bin = xr.concat(bin_list, dim = "DataArray")

# now we need to add the enclosure
test_4_bin

    
    
test_4_bin_list = []
for i in range(0, (len_data)):
    final_bin = np.array(test_4_bin[i])

    for i in range(0, (np.size(final_bin, 1))):
        final_bin[0, i] = 0
        final_bin[- 1, i] = 0
        final_bin[i, 0] = 0
        final_bin[i, -1] = 0
        final_bin_xr = final_bin.astype(np.int)
        final_bin_xr = xr.DataArray(final_bin_xr)
    test_4_bin_list.append(final_bin_xr)

test_4_bin_final = xr.concat(test_4_bin_list, dim = "DataArray")


len(test_4_bin_final)

1

In [7]:
zero_counter_list = []
one_counter_list = []
for i in range(0, (len_data)):

    img_arr = np.array(test_4_bin_final[i])
    
    # Set "connectivity" to 1 for 4-connection, 2 for 8-connection (one's and zero's should have diff values for 
    # connectivity)
    one_labels = label(img_arr, connectivity=1 , background=-1)  

    one_label_vals = np.unique(one_labels)                      
    one_counter = 0
    for i in one_label_vals:
        one_indices = np.where(one_labels == i)
        if one_indices:
            if img_arr[one_indices][0] == 0:
            
                one_counter += 1
    one_counter_list.append(one_counter)
    
    img_arr_inv = img_arr
    for i in range(0, (np.size(img_arr, 1))):
        for j in range(0, (np.size(img_arr, 1))):
            if img_arr_inv[i][j] == 0:
                img_arr_inv[i][j] = 1
            elif img_arr_inv[i][j] == 1:
                img_arr_inv[i][j] = 0
                
                
    # now run the same counter on the inverse matrix (to get number of connected black regions)
    
    # Set "connectivity" to 1 for 4-connection, 2 for 8-connection
    zero_labels = label(img_arr_inv, connectivity=2, background=-1)  
    zero_label_vals = np.unique(zero_labels)                      
    zero_counter = 0
    for i in zero_label_vals:
        zero_indices = np.where(zero_labels == i)
        if zero_indices:
            if img_arr_inv[zero_indices][0] == 0:
            #print('hole: ', indices)
                zero_counter += 1
    
    zero_counter_list.append(zero_counter)
    
counter_list = list(map(operator.sub, one_counter_list, zero_counter_list))
    
    
print("count of connected 0's: " + str(one_counter_list))
print("count of connected 1's: " + str(zero_counter_list))
print("Euler Characteristic: " + str(counter_list))

count of connected 0's: [5]
count of connected 1's: [3]
Euler Characteristic: [2]


In [8]:
## Run below code if saving dataarray to file
import netCDF4

#new_filename_3 = './data_iter3.nc'
#print ('saving to ', new_filename_3)

#test_4_bin_final.to_netcdf(path=new_filename_3)
#print ('finished saving')

# Part 2: Building the networks in tree

In [9]:
from pylab import imshow, show, get_cmap
from numpy import random
import cv2 as cv
import numpy as np
from numpy import linalg
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from PIL import Image
import xarray as xr
import collections
import tensorflow as tf
import netCDF4
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [10]:
# Load large dataset for first iteration of tree
data_1 = xr.open_dataarray("data_iter3.nc")
data_1 = np.array(data_1)
nn_data = pd.DataFrame()
for i in range(0, len(data_1)):
    row = data_1[i].flatten()
    row = pd.Series(row)
 
    nn_data = nn_data.append(row, ignore_index=True)
    
labels = np.load("labels_iter3.npy")

In [ ]:
# append labels to data
labels = pd.Series(labels)
nn_data = pd.concat([nn_data, labels], axis = 1)

In [ ]:
names = nn_data[ (nn_data.iloc[:, -1] > 3) | (nn_data.iloc[:, -1] < -4)  ].index
nn_data2 = nn_data
nn_data2.drop(names , inplace=True)
nn_data2 = nn_data2.dropna()
nn_data2.isnull().values.any()

In [ ]:
nn_data2_bin = []
for i in range(0, len(nn_data2)):
    if nn_data2.iloc[i, -1] < 0:
        nn_data2_bin.append(0)
    else:
        nn_data2_bin.append(1)
nn_data2['binary_label'] = nn_data2_bin

In [ ]:
X = nn_data2.iloc[:,0:144]
Y = nn_data2.iloc[:, -1]
X = np.array(X)
Y = np.array(Y)

In [ ]:
## First Model
model_1 = Sequential()
model_1.add(Dense(250, input_dim=144, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(95, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1, activation = "sigmoid"))

rms = optimizers.RMSprop(learning_rate=0.0001, rho=0.9)
model_1.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, Y, train_size=0.70,
                                                                    test_size=0.30)


class_weights_1 = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train1),
                                                 y_train1)

In [ ]:
model_1.fit(np.array(X_train1), np.array(y_train1) ,validation_data=(np.array(X_test1),np.array(y_test1)),
            class_weight = class_weights_1, epochs=150, batch_size=100)

In [ ]:
scores = model_1.evaluate(X_test1, y_test1, verbose=0)
print("%s: %.2f%%" % (model_1.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_1.save("model_1.h5")
print("Saved model to disk")

In [ ]:
row2_a = pd.read_csv("row2_a.csv")
row2_b = pd.read_csv("row2_b.csv")

In [ ]:
# Data for row2_a (values are between -6 and -1)

X_2a = row2_a.iloc[:,1:145]
Y_2a = row2_a.iloc[:, -1]
X_2a = np.array(X_2a)
Y_2a = np.array(Y_2a)

# Data for row2_b (values are between 0 and 5)

X_2b = row2_b.iloc[:,1:145]
Y_2b = row2_b.iloc[:, -1]
X_2b = np.array(X_2b)
Y_2b = np.array(Y_2b)

In [ ]:
# Models for second layer of tree

model_2a = Sequential()
model_2a.add(Dense(200, input_dim=144, activation='relu'))
model_2a.add(Dropout(0.3))
model_2a.add(Dense(90, activation='relu'))
#model_2a.add(Dropout(0.2))
model_2a.add(Dense(30, activation='relu'))
model_2a.add(Dropout(0.2))
model_2a.add(Dense(1, activation = "sigmoid"))

opt = optimizers.SGD(lr=0.0001)
rms = optimizers.RMSprop(learning_rate=0.0005, rho=0.9)
model_2a.compile(loss='binary_crossentropy', optimizer= rms, metrics=['accuracy'])

model_2b = Sequential()
model_2b.add(Dense(200, input_dim=144, activation='relu'))
model_2b.add(Dropout(0.3))
model_2b.add(Dense(90, activation='relu'))
model_2b.add(Dropout(0.2))
model_2b.add(Dense(40, activation='relu'))
model_2b.add(Dropout(0.2))
model_2b.add(Dense(1, activation = "sigmoid"))

opt = optimizers.SGD(lr=0.0001)
rms = optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model_2b.compile(loss='binary_crossentropy', optimizer= "adam", metrics=['accuracy'])

In [ ]:
X_train_2a, X_test_2a, y_train_2a, y_test_2a = train_test_split(X_2a, Y_2a, train_size=0.60,
                                                                    test_size=0.40)

X_train_2b, X_test_2b, y_train_2b, y_test_2b = train_test_split(X_2b, Y_2b, train_size=0.70,
                                                                    test_size=0.30)

In [ ]:
class_weights_2a = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_2a),
                                                 y_train_2a)

class_weights_2b = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_2b),
                                                 y_train_2b)

class_weight_2a = {0:75, 1:24}

class_weight_2b = {0:32.4, 1:67.5}

In [ ]:
model_2a.fit((X_train_2a), (y_train_2a) ,validation_data=((X_test_2a),(y_test_2a)),  class_weight=class_weight_2a,
                                                                  epochs=200, batch_size=100)

In [ ]:
vals = model_2a.predict_classes(X_test_2a)
test_2a = pd.DataFrame(y_test_2a)
test_2a['predict'] = vals

In [ ]:
count = 0
for i in range(0, len(test_2a)):
    if test_2a.iloc[i, 0] == test_2a.iloc[i, 1]:
        count += 1
        
print(count)

In [ ]:
model_2b.fit((X_train_2b), (y_train_2b) ,validation_data=((X_test_2b),(y_test_2b)), class_weight=class_weight_2b, 
                                                                  epochs=150, batch_size=200)

In [ ]:
scores = model_2a.evaluate(X_test_2a, y_test_2a, verbose=0)
print("%s: %.2f%%" % (model_2a.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_2a.save("model_2a.h5")
print("Saved model to disk")

In [ ]:
scores = model_2b.evaluate(X_test_2b, y_test_2b, verbose=0)
print("%s: %.2f%%" % (model_2b.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_2b.save("model_2b.h5")
print("Saved model to disk")

In [ ]:
# Need to one_hot encode these dataframes
row3_a = pd.read_csv("row3_a.csv")
row3_b = pd.read_csv("row3_b.csv")
row3_c = pd.read_csv("row3_c.csv")
row3_d = pd.read_csv("row3_d.csv")

In [ ]:
row_3a_final = np.array(row_3a_final)
row3_b = np.array(row3_b)
row3_c = np.array(row3_c)
row_3d_final = np.array(row_3d_final)

row_3_test = np.concatenate((row_3a_final, row3_b, row3_c, row_3d_final), axis = 0)
row_3_test = pd.DataFrame(row_3_test)
row_3_test

In [ ]:
index_3a = row_3_test.iloc[:, -1].between(-4, -3, inclusive = True) 
index_3b = row_3_test.iloc[:, -1].between(-2, -1, inclusive = True) 
index_3c = row_3_test.iloc[:, -1].between(0, 1, inclusive = True) 
index_3d = row_3_test.iloc[:, -1].between(2, 3, inclusive = True) 



row_3a_s = row_3_test[index_3a]
row_3b_s = row_3_test[index_3b]
row_3c_s = row_3_test[index_3c]
row_3d_s = row_3_test[index_3d]



In [ ]:
# Transform x and y for smaller subset

# Row 3a (add 4 bc negative outcomes)
X_3a = row_3a_s.iloc[:,0:144]
Y_3a = row_3a_s.iloc[:, -1]
X_3a = np.array(X_3a)
Y_3a = np.array(Y_3a) + 4
Y_3a = Y_3a.astype(int)

# Row 3b (add 2 bc negative outcomes)

X_3b = row_3b_s.iloc[:,0:144]
Y_3b = row_3b_s.iloc[:, -1]
X_3b = np.array(X_3b)
Y_3b = np.array(Y_3b) + 2
Y_3b = Y_3b.astype(int)

# Row 3c 
X_3c = row_3c_s.iloc[:,0:144]
Y_3c = row_3c_s.iloc[:, -1]
X_3c = np.array(X_3c)
Y_3c = np.array(Y_3c) 
Y_3c = Y_3c.astype(int)

# Row 3d (to start from 0)
X_3d = row_3d_s.iloc[:,0:144]
Y_3d = row_3d_s.iloc[:, -1]
X_3d = np.array(X_3d)
Y_3d = np.array(Y_3d) 
Y_3d = Y_3d.astype(int) - 2

In [ ]:
# Train models in row 3

model_3a = Sequential()
model_3a.add(Dense(250, input_dim=144, activation='relu'))
#model_3a.add(Dropout(0.2))
#model_3a.add(Dense(95, activation='relu'))
#model_3a.add(Dropout(0.2))
#model_3a.add(Dense(60, activation='relu'))
#model_3a.add(Dropout(0.2))
#model_3a.add(Dense(20, activation='relu'))
model_3a.add(Dense(1, activation = "sigmoid"))

rms = optimizers.RMSprop(learning_rate=0.0005, rho=0.9)
model_3a.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])


# Row 2
model_3b = Sequential()
model_3b.add(Dense(250, input_dim=144, activation='relu'))
#model_3b.add(Dropout(0.2))
#model_3b.add(Dense(95, activation='relu'))
#model_3b.add(Dropout(0.2))
#model_3b.add(Dense(60, activation='relu'))
model_3b.add(Dropout(0.2))
model_3b.add(Dense(1, activation = "sigmoid"))

model_3b.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Row 3
model_3c = Sequential()
model_3c.add(Dense(250, input_dim=144, activation='relu'))
model_3c.add(Dropout(0.2))
model_3c.add(Dense(95, activation='relu'))
model_3c.add(Dropout(0.2))
model_3c.add(Dense(60, activation='relu'))
model_3c.add(Dropout(0.2))
model_3c.add(Dense(1, activation = "sigmoid"))

model_3c.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Row 4
model_3d = Sequential()
model_3d.add(Dense(250, input_dim=144, activation='relu'))
model_3d.add(Dropout(0.2))
model_3d.add(Dense(95, activation='relu'))
model_3d.add(Dropout(0.2))
model_3d.add(Dense(60, activation='relu'))
model_3d.add(Dropout(0.2))
model_3d.add(Dense(1, activation = "sigmoid"))

model_3d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train_3a, X_test_3a, y_train_3a, y_test_3a = train_test_split(X_3a, Y_3a, train_size=0.60,
                                                                    test_size=0.40)


#weights
class_weights_3a = class_weight.compute_class_weight('balanced', np.unique(y_train_3a), y_train_3a)



X_train_3b, X_test_3b, y_train_3b, y_test_3b = train_test_split(X_3b, Y_3b, train_size=0.60,
                                                                    test_size=0.40)


#weights
class_weights_3b = class_weight.compute_class_weight('balanced', np.unique(y_train_3b), y_train_3b)


X_train_3c, X_test_3c, y_train_3c, y_test_3c = train_test_split(X_3c, Y_3c, train_size=0.60,
                                                                    test_size=0.40)

#weights
class_weights_3c = class_weight.compute_class_weight('balanced', np.unique(y_train_3c), y_train_3c)


X_train_3d, X_test_3d, y_train_3d, y_test_3d = train_test_split(X_3d, Y_3d, train_size=0.60,
                                                                    test_size=0.40)

# weights
class_weights_3d = class_weight.compute_class_weight('balanced', np.unique(y_train_3d), y_train_3d)



In [ ]:
model_3a.fit(X_train_3a, y_train_3a ,validation_data=(X_test_3a,y_test_3a), epochs=50, batch_size=200, 
             class_weight = class_weights_3a)

In [ ]:
model_3b.fit(X_train_3b, y_train_3b ,validation_data=(X_test_3b,y_test_3b), epochs=50, batch_size=200,
            class_weight = class_weights_3b)

In [ ]:
model_3c.fit(X_train_3c, y_train_3c ,validation_data=(X_test_3c,y_test_3c), epochs=50, batch_size=200,
            class_weight = class_weights_3c)

In [ ]:
model_3d.fit(X_train_3d, y_train_3d ,validation_data=(X_test_3d,y_test_3d), epochs=50, batch_size=200,
            class_weight = class_weights_3d)

In [ ]:
scores = model_3a.evaluate(X_test_3a, y_test_3a, verbose=0)
print("%s: %.2f%%" % (model_3a.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_3a.save("model_3a.h5")
print("Saved model to disk")

In [ ]:
scores = model_3b.evaluate(X_test_3b, y_test_3b, verbose=0)
print("%s: %.2f%%" % (model_3b.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_3b.save("model_3b.h5")
print("Saved model to disk")

In [ ]:
scores = model_3c.evaluate(X_test_3c, y_test_3c, verbose=0)
print("%s: %.2f%%" % (model_3c.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_3c.save("model_3c.h5")
print("Saved model to disk")

In [ ]:
scores = model_3d.evaluate(X_test_3d, y_test_3d, verbose=0)
print("%s: %.2f%%" % (model_3d.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model_3d.save("model_3d.h5")
print("Saved model to disk")

# Implement Deep Nested Neural Tree

In [ ]:
model_1 = tf.keras.models.load_model('model_1.h5')
model_2a = tf.keras.models.load_model('model_2a.h5')
model_2b = tf.keras.models.load_model('model_2b.h5')
model_3a = tf.keras.models.load_model('model_3a.h5')
model_3b = tf.keras.models.load_model('model_3b.h5')
model_3c = tf.keras.models.load_model('model_3c.h5')
model_3d = tf.keras.models.load_model('model_3d.h5')

In [ ]:
test_dataset = pd.read_csv("test_final.csv")
test_dataset = test_dataset.iloc[:, 1:146]

In [ ]:
test_x = test_dataset.iloc[:, 0:144]
test_y = test_dataset.iloc[:, -1]
test_x